<a href="https://colab.research.google.com/github/ashish2179/MachineLearning/blob/master/pubg_kmean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install -U -q pyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#data dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16,10]
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import xgboost as xgb

%matplotlib inline
plt.rcParams['axes.unicode_minus']=False

    100% |████████████████████████████████| 993kB 19.7MB/s 


In [0]:
#authentication

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#csvfile_id = 1LBuQLnc1S5ccGGnL1x0ACDGWoW3w_Bj5

pubg_data = drive.CreateFile({'id': '1LBuQLnc1S5ccGGnL1x0ACDGWoW3w_Bj5'})
pubg_data.GetContentFile('pubg.csv')

data = pd.read_csv('pubg.csv')
data.head()


,tracker_id,solo_KillDeathRatio,solo_WinRatio,solo_RoundsPlayed,solo_Wins,solo_WinTop10Ratio,solo_Top10s,solo_Top10Ratio,solo_Losses,solo_DamagePg,solo_HeadshotKillsPg,solo_HealsPg,solo_KillsPg,solo_MoveDistancePg,solo_TimeSurvivedPg,solo_Top10sPg,solo_Kills,solo_Assists,solo_HeadshotKillRatio,solo_AvgSurvivalTime
0,34359,0.54,0.48,1681,8,0.05,114,6.8,1673,76.92,0.10,1.37,0.54,2017.76,715.67,0.07,911,91,0.18,871.71
1,70832,1.37,0.71,1552,11,0.08,125,8.1,1541,155.33,0.23,1.34,1.36,1943.60,631.10,0.08,2106,87,0.17,466.83
2,77162,0.53,0.26,1545,4,0.02,98,6.3,1541,69.65,0.09,0.79,0.53,1537.46,655.30,0.06,823,58,0.16,470.44
3,41250,0.87,0.78,1541,12,0.11,130,8.4,1529,117.32,0.17,1.17,0.86,2033.91,806.88,0.08,1332,105,0.20,880.84
4,168294,0.50,0.00,1537,0,0.00,80,5.2,1537,67.11,0.14,1.40,0.50,2534.69,750.92,0.05,761,83,0.28,565.15


In [0]:
traindata = data.iloc[:,1:]
traindata.head()

,solo_KillDeathRatio,solo_WinRatio,solo_RoundsPlayed,solo_Wins,solo_WinTop10Ratio,solo_Top10s,solo_Top10Ratio,solo_Losses,solo_DamagePg,solo_HeadshotKillsPg,solo_HealsPg,solo_KillsPg,solo_MoveDistancePg,solo_TimeSurvivedPg,solo_Top10sPg,solo_Kills,solo_Assists,solo_HeadshotKillRatio,solo_AvgSurvivalTime
0,0.54,0.48,1681,8,0.05,114,6.8,1673,76.92,0.10,1.37,0.54,2017.76,715.67,0.07,911,91,0.18,871.71
1,1.37,0.71,1552,11,0.08,125,8.1,1541,155.33,0.23,1.34,1.36,1943.60,631.10,0.08,2106,87,0.17,466.83
2,0.53,0.26,1545,4,0.02,98,6.3,1541,69.65,0.09,0.79,0.53,1537.46,655.30,0.06,823,58,0.16,470.44
3,0.87,0.78,1541,12,0.11,130,8.4,1529,117.32,0.17,1.17,0.86,2033.91,806.88,0.08,1332,105,0.20,880.84
4,0.50,0.00,1537,0,0.00,80,5.2,1537,67.11,0.14,1.40,0.50,2534.69,750.92,0.05,761,83,0.28,565.15


In [0]:
dataset_array = traindata.values
print(dataset_array.dtype)
print(dataset_array)

float64
[[5.4000e-01 4.8000e-01 1.6810e+03 ... 9.1000e+01 1.8000e-01 8.7171e+02]
 [1.3700e+00 7.1000e-01 1.5520e+03 ... 8.7000e+01 1.7000e-01 4.6683e+02]
 [5.3000e-01 2.6000e-01 1.5450e+03 ... 5.8000e+01 1.6000e-01 4.7044e+02]
 ...
 [9.3000e-01 4.0000e-01 2.4700e+02 ... 1.2000e+01 2.6000e-01 6.8339e+02]
 [1.1600e+00 1.6200e+00 2.4700e+02 ... 2.0000e+01 1.9000e-01 8.1454e+02]
 [1.6800e+00 3.6400e+00 2.4700e+02 ... 2.2000e+01 2.3000e-01 8.9764e+02]]


In [0]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(dataset_array)
y_kmean = kmeans.predict(dataset_array)